In [ ]:
import hvplot.pandas
import holoviews as hv
import pandas as pd
import geopandas as gp
import geoviews as gv
import shapely
import panel as pn
from shapely.geometry import Polygon, LineString, Point
# from pyposeidon.utils.cfl import parse_hgrid
from __future__ import annotations

import collections
import io
import itertools
import os
import typing as T

import numpy as np
import numpy.typing as npt

VOSPOROS_SOUTH = shapely.Polygon([
    (26.77069525861703, 40.44054922299234),
    (26.563328315257657, 40.4133689586624),
    (26.115328982567448, 39.99024357789592),
    (26.394107058739323, 39.98182584129004),
    (26.785494998192448, 40.342851238592424),
    (26.77069525861703, 40.44054922299234),
])
def _readline(fd: bytes) -> bytes:
    return fd.readline().split(b"=")[0].split(b"!")[0].strip()


def parse_hgrid(
    path: os.PathLike[str] | str,
    include_boundaries: bool = False,
    sep: str | None = None,
) -> dict[str, T.Any]:
    """
    Parse an hgrid.gr3 file.

    The function is also able to handle fort.14 files, too, (i.e. ADCIRC)
    but the boundary parsing is not keeping all the available information.
    """
    rvalue: dict[str, T.Any] = {}
    with open(path, "rb") as fd:
        _ = fd.readline()  # skip line
        no_elements, no_points = map(int, fd.readline().strip().split(b"!")[0].split())
        nodes_buffer = io.BytesIO(b"\n".join(itertools.islice(fd, 0, no_points)))
        nodes = np.loadtxt(nodes_buffer, delimiter=sep, usecols=(1, 2, 3))
        elements_buffer = io.BytesIO(b"\n".join(itertools.islice(fd, 0, no_elements)))
        elements = np.loadtxt(elements_buffer, delimiter=sep, usecols=(2, 3, 4), dtype=int)
        elements -= 1  # 0-based index for the nodes
        rvalue["nodes"] = nodes
        rvalue["elements"] = elements
        # boundaries
        if include_boundaries:
            boundaries = collections.defaultdict(list)
            no_open_boundaries = int(_readline(fd))
            total_open_boundary_nodes = int(_readline(fd))
            for i in range(no_open_boundaries):
                no_nodes_in_boundary = int(_readline(fd))
                boundary_nodes = np.genfromtxt(fd, delimiter=sep, usecols=(0,), max_rows=no_nodes_in_boundary, dtype=int)
                boundaries["open"].append(boundary_nodes - 1)  # 0-based index
            # closed boundaries
            no_closed_boundaries = int(_readline(fd))
            total_closed_boundary_nodes = int(_readline(fd))
            for _ in range(no_closed_boundaries):
                # Sometimes it seems that the closed boundaries don't have a "type indicator"
                # For example: Test_COSINE_SFBay/hgrid.gr3
                # In this cases we assume that boundary type is 0 (i.e. land in schism)
                # XXX Maybe check the source code?
                parsed = _readline(fd).split(b" ")
                if len(parsed) == 1:
                    no_nodes_in_boundary = int(parsed[0])
                    boundary_type = 0
                else:
                    no_nodes_in_boundary, boundary_type = map(int, parsed)
                boundary_nodes = np.genfromtxt(fd, delimiter=sep, usecols=(0,), max_rows=no_nodes_in_boundary, dtype=int)
                boundary_nodes -= 1  # 0-based-index
                boundaries[boundary_type].append(boundary_nodes)
            rvalue["boundaries"] = boundaries
    return rvalue

def extract_area(x: np.ndarray, y: np.ndarray, triangles: np.ndarray, lon_range: tuple[float, float], lat_range: tuple[float, float]):
    mask = (x >= lon_range[0]) & (x <= lon_range[1]) & \
           (y >= lat_range[0]) & (y <= lat_range[1])
    node_indices = np.where(mask)[0]
    node_map = {old: new for new, old in enumerate(node_indices)}
    node_map_reverse = {new: old for new, old in enumerate(node_indices)}
    extracted_x = x[node_indices]
    extracted_y = y[node_indices]
    triangle_indices = np.array([idx for idx, tri in enumerate(triangles) if all(i in node_map for i in tri)])
    extracted_triangles = triangles[triangle_indices]
    extracted_triangles = np.array([[node_map[i] for i in tri] for tri in extracted_triangles])
    
    return extracted_x, extracted_y, extracted_triangles, node_indices, triangle_indices, node_map_reverse

# 2. Convert triangulation to GeoDataFrame with MultiPolygons
def tri_to_geopandas(points, tris, values):
    """Convert triangulation to GeoDataFrame with values for coloring"""
    polygons = [Polygon(points[tri]) for tri in tris]

    gdf = gp.GeoDataFrame({
        'A': values[tris[:,0]],
        'B': values[tris[:,1]],
        'C': values[tris[:,2]],
        'geometry': polygons
        },
        crs="EPSG:4326"  # using WGS84 for demonstration
    )    
    return gdf

def drop(nodes, elems, drop_indices):
    drop_indices = np.unique(drop_indices)
    keep_mask = np.ones(len(nodes), dtype=bool)
    keep_mask[drop_indices] = False
    nodes = nodes[keep_mask]
    old_to_new = -np.ones(len(keep_mask), dtype=int)
    old_to_new[np.where(keep_mask)[0]] = np.arange(len(nodes))
    mask_valid = np.all(np.isin(elems, np.where(keep_mask)[0]), axis=1)
    elems = elems[mask_valid]
    elems = old_to_new[elems]
    return nodes, elems

def render_gdf(gdf, case = "global"):
    if case == 'global':
        color = 'red'
    elif case == 'black sea': 
        color = "blue"
    else:
        raise ValueError("case has to be 'global' or 'black sea'!")

    if isinstance(gdf, gp.GeoDataFrame):
        return gdf.hvplot(geo=True, tiles=True, c=color, label=case)
    elif isinstance(gdf, pd.DataFrame):
        return gdf.hvplot.points(geo=True, tiles=True, c=color, line_color='k', tools=["box_select"], hover_cols = ['index'], label=case)
    

def subset_gdf(gdf, poly: shapely.Polygon):
    return gdf[gdf.geometry.within(poly)]

def polygon_to_points(gdf):
    coords_index = []
    for g in gdf.iterrows(): 
        coords_index.append([*g[1].geometry.exterior.xy, np.array([g[1].A,g[1].B,g[1].C,g[1].A])])
    pp = np.array(coords_index).transpose(0, 2, 1).reshape(-1, 3)
    return pd.DataFrame(pp, columns=['lon', 'lat', 'index'])

In [ ]:
file = "/home/tomsail/Documents/work/python/seareport_org/seareport_models/v3.2/GSHHS_f_0.01_final.gr3"
mesh_dic = parse_hgrid(file)

In [ ]:
x, y, depth = mesh_dic['nodes'].T
tris = mesh_dic["elements"]

subset global mesh to vosphoros south strait

In [ ]:
x_, y_, tri_, ind_, tri_sub, node_mapping = extract_area(x,y,tris, (25,45), (40,48))
gdf_ = tri_to_geopandas(np.vstack((x_, y_)).T,tri_, ind_)
global_vosphoros_gdf = subset_gdf(gdf_, VOSPOROS_SOUTH)
global_vosphoros_points = polygon_to_points(global_vosphoros_gdf)

In [ ]:
points_ = render_gdf(global_vosphoros_points)
global_vosphoros_ = render_gdf(global_vosphoros_gdf)
(global_vosphoros_ * points_).opts(width = 1000, height=800)

In [ ]:
from pyposeidon.mgmsh import read_msh

black_sea_mesh = read_msh("out_blacksea_0.2.msh")
bs_pts = np.vstack((black_sea_mesh.SCHISM_hgrid_node_x, black_sea_mesh.SCHISM_hgrid_node_y)).T
bs_tri = black_sea_mesh.SCHISM_hgrid_face_nodes.data
black_sea_mesh

In [ ]:
gdf_ = tri_to_geopandas(bs_pts, bs_tri, np.arange(len(bs_pts)))
black_sea_vosporos_gdf = subset_gdf(gdf_, VOSPOROS_SOUTH)
black_sea_vosporos_points = polygon_to_points(black_sea_vosporos_gdf)
black_sea_vosporos_ = render_gdf(black_sea_vosporos_gdf, "black sea")

In [ ]:
selection = hv.streams.Selection1D(source=points_)
# Create a function to print selected points
def print_all_points(df: pd.DataFrame, indices: pd.Index[int], text_box: pn.widgets.TextAreaInput) -> T.Any:
    print("Selected indices:", indices)
    if indices:
        ind = np.unique(indices)
        list = df["index"].iloc[ind].astype(int).unique()
        value = ",\n".join(map(str, list))
    else:
        value = "No selection!"
    text_box.value = value

points_all = pn.widgets.TextAreaInput(value="", height=200, placeholder="Selected indices will appear here")  # type: ignore[no-untyped-call]
selection.add_subscriber(lambda index: print_all_points(df=global_vosphoros_points, indices=index, text_box=points_all))

plot_ = (black_sea_vosporos_ * global_vosphoros_ * points_).opts(width = 700, height=800)
layout = pn.Column(
    points_.opts(width=900, height=500),
    pn.Row(
        pn.Column("## Selected Indices:", points_all),
    ),
)    
out = pn.panel(layout).servable()
out

we'll fix the following points and regenerate the meshing for the black sea

In [ ]:
pts_ind = [6817898,
6697291,
6549074,
1714532,
1473386]
coords = mesh_dic['nodes'][pts_ind][:,:2]
gdf = gp.GeoDataFrame(
    pd.DataFrame(coords, columns=["lon", "lat"]),
    geometry=[Point(xy) for xy in coords],
    crs="EPSG:4326"
)
gdf['physical'] = "force"
gdf
gdf.to_file("points_4326.shp", driver='ESRI Shapefile')

all these points need to be suppressed

In [ ]:
global_nodes = np.vstack((x,y)).T

In [ ]:
discard = [1807639,456464,4792637,2451328,39451,5503929,4015013,6409047,2134299,4386548,4967757,
4856350,7851069,1324320,3643482,7452558,512577,6384066,677206,2528912,7294189,
167240,5582097,7382856,4192983,7134380,1647070,7700285,7907239,2334291,3421735,363349,
7046390,5460790,1145947,1314347,2426245,5698280,3282119,4838108,332392,6336320,1580434,
4864137,2614067,263530,1928581,1756355,4590953,47220,4869691,3173761,7342931,551330,189622,
2312122,7482244,5405589,4401906]

nno, ell = drop(global_nodes, tris, discard)

In [ ]:
llo, lla, tri_indices, node_indices, tri_sub, node_mapping = extract_area(nno[:,0],nno[:,1],ell, (25,45), (40,48))
global_black_sea_gdf = tri_to_geopandas(np.vstack((llo,lla)).T, tri_indices, node_indices)
global_vosphoros_gdf = global_black_sea_gdf[global_black_sea_gdf.geometry.within(VOSPOROS_SOUTH)]
global_vosphoros_ = render_gdf(global_vosphoros_gdf)

In [ ]:
# now create points from the black sea mesh
points2_ = render_gdf(black_sea_vosporos_points, "black sea")

In [ ]:
selection = hv.streams.Selection1D(source=points2_)
points_all = pn.widgets.TextAreaInput(value="", height=200, placeholder="Selected indices will appear here")  # type: ignore[no-untyped-call]
def print_all_points(df: pd.DataFrame, indices: pd.Index[int], text_box: pn.widgets.TextAreaInput) -> T.Any:
    print("Selected indices:", indices)
    if indices:
        ind = np.unique(indices)
        list = df["index"].iloc[ind].astype(int).unique()
        value = ",\n".join(map(str, list))
    else:
        value = "No selection!"
    text_box.value = value

selection.add_subscriber(lambda index: print_all_points(df=black_sea_vosporos_points, indices=index, text_box=points_all))
plot_ = (black_sea_vosporos_ * global_vosphoros_ * points_ * points2_).opts(width = 700, height=800)
layout = pn.Column(
    plot_.opts(width=900, height=500),
    pn.Row(
        pn.Column("## Selected Indices:", points_all),
    ),
)    
out = pn.panel(layout).servable()
out

here are the following node to drop on the back sea mesh

In [ ]:
discard2 = [13598,39688,31622,28962,28963,528,529,522,28379,17111,29402,37351,23225,17568,45282,46776,
           523,46587,31245,46194,48307,524,47748,44,48416,519,518,42,520,525,521,527,526,35710,43,149, 145]

new_bs_pts, new_bs_tri = drop(bs_pts,bs_tri, discard2)

In [ ]:
new_black_sea_gdf = tri_to_geopandas(new_bs_pts, new_bs_tri, np.arange(len(new_bs_pts)))
new_bs_vosphoros_gdf = new_black_sea_gdf[new_black_sea_gdf.geometry.within(VOSPOROS_SOUTH)]
new_black_sea_vosporos_points = polygon_to_points(new_bs_vosphoros_gdf)
new_points2_ = render_gdf(new_black_sea_vosporos_points, "black sea")

In [ ]:
plot_ = (render_gdf(global_vosphoros_gdf) * 
         render_gdf(new_bs_vosphoros_gdf, case = "black sea") * points_ * new_points2_).opts(width = 700, height=800)

In [ ]:
plot_

In [ ]:
stitching_map = dict([
    [512, 7732209],
    [144, 1714532],
    [143, 6549074],
    [142, 6697291],
    [37476, 3418943],
    [22941, 1473386]
])

In [ ]:
import numpy as np

def stitch_meshes(global_nodes, global_elements, bs_pts, bs_tri, stitching_map):
    mapped_bs_indices = set(stitching_map.keys())
    unmapped_bs_indices = [i for i in range(len(bs_pts)) if i not in mapped_bs_indices]

    index_mapping = {}
    new_nodes = global_nodes.tolist()

    for i in unmapped_bs_indices:
        new_index = len(new_nodes)
        index_mapping[i] = new_index
        # Insert with dummy z-value (can be interpolated or set to 0)
        new_nodes.append([bs_pts[i][0], bs_pts[i][1], 0.0])

    for bs_idx, global_idx in stitching_map.items():
        index_mapping[bs_idx] = global_idx

    new_elements = []
    for tri in bs_tri:
        new_tri = [index_mapping[idx] for idx in tri]
        new_elements.append(new_tri)

    all_elements = np.vstack([global_elements, np.array(new_elements, dtype=int)])

    return np.array(new_nodes), all_elements


In [ ]:
stiched_nodes, stiched_tri = stitch_meshes(mesh_dic['nodes'], mesh_dic['elements'], new_bs_pts, new_bs_tri, stitching_map)
stiched_nodes_fix, stiched_tri_fix = drop(stiched_nodes, stiched_tri, discard)

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (10,6))
ax.triplot(stiched_nodes_fix[:,0], stiched_nodes_fix[:,1], stiched_tri_fix, lw=0.2)
ax.set_xlim([26,30])
ax.set_ylim([39.75,41.5])

In [ ]:
import meshio

points = np.column_stack((stiched_nodes_fix[:,0], stiched_nodes_fix[:,1], np.zeros(len(stiched_nodes_fix))))  # Add z=0
cells = [("triangle", stiched_tri_fix)]
mesh = meshio.Mesh(points=points, cells=cells)
mesh.write("output_mesh.vtk")

In [ ]:
from xarray_selafin.xarray_backend import SelafinAccessor
import xarray as xr

ds = xr.Dataset({
    "B": (("time", "node"), np.zeros((1,len(stiched_nodes_fix)))),
    },
    coords = {
        "x" : ("node", stiched_nodes_fix[:,0]),
        "y" : ("node", stiched_nodes_fix[:,1]),
        "time": [pd.Timestamp.now()]
    } )
ds.attrs['ikle2'] = stiched_tri_fix + 1
ds

In [ ]:
ds.selafin.write('out.slf')